In [1]:
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime
import gc
import os
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
## read files
df_class_train = pd.read_csv('./datasets/class/df_class_train.csv')
df_class_test = pd.read_csv('./datasets/class/df_class_test.csv')

In [10]:
df_class_train['average'] = (df_class_train['lgb_outlier_pred'] + df_class_train['cat_outlier_pred'] + df_class_train['lgb_classifier_outlier_preds'] + df_class_train['outlier_pred'])/4
df_class_test['average'] = (df_class_test['lgb_outlier_pred'] + df_class_test['cat_outlier_pred'] + df_class_test['lgb_classifier_outlier_preds'] + df_class_test['outlier_pred'])/4

In [3]:
NFOLDS = 5
# kfold = KFold(n_splits=NFOLDS,shuffle=False,random_state=5)
kfold = StratifiedKFold(n_splits=NFOLDS,shuffle=False,random_state=20)
label = ['is_outlier']
tr_features = [_f for _f in df_class_train.columns if _f not in ['card_id','is_outlier']]

ntrain = df_class_train.shape[0]
ntest = df_class_test.shape[0]
oof_train_pred = np.zeros((ntrain,))
oof_test_pred = np.zeros((ntest,))
oof_test_pred_skf = np.empty((NFOLDS, ntest))
for foldIndex,(dev_index,val_index) in enumerate(kfold.split(df_class_train,df_class_train['is_outlier'])):
    clf = LogisticRegression(class_weight='balanced',random_state=42)
    x_dev = df_class_train[tr_features].iloc[dev_index]
    y_dev = df_class_train[label].iloc[dev_index]
    x_val = df_class_train[tr_features].iloc[val_index]
    y_val = df_class_train[label].iloc[val_index]
    clf.fit(x_dev.values,y_dev.values)
    oof_test_pred_skf[foldIndex,:] = clf.predict_proba(df_class_test[tr_features].values)[:,1]
    oof_train_pred[val_index] = clf.predict_proba(x_val.values)[:,1]

oof_test_pred[:] = oof_test_pred_skf.mean(axis=0)

score = np.sqrt((np.sum(np.square(oof_train_pred - df_class_train[label].values.reshape(-1,)))/ntrain))

df_Log_sub = pd.DataFrame(data=df_class_test['card_id'].values,columns=['card_id'])
df_Log_sub['log_outlier_pred'] = oof_test_pred
print("score = %s"%score)

df_Log_sub.sort_values(by=['log_outlier_pred'],ascending=False,inplace=True)

score = 0.34704594033443975


#### selected abnormal

In [38]:
topk = 150
outlier_cols = [_f for _f in df_class_train.columns if _f not in ['card_id','is_outlier']]

aggs = set(df_class_train.card_id)

for col in outlier_cols:
    df = df_class_train.sort_values(by=col,ascending=False)[['card_id',col]].iloc[:topk,]
    aggs = set(aggs) & set(df['card_id'])

df = df_class_train[df_class_train['card_id'].isin(list(aggs))][['card_id','is_outlier']]
accuracy = df[df.is_outlier==1].shape[0]/len(aggs)

print(len(aggs))
print("Accuracy:%s"%accuracy)

9
Accuracy:0.5555555555555556


In [39]:
df_class_train[df_class_train['card_id'].isin(list(aggs))]

,card_id,is_outlier,lgb_outlier_pred,cat_outlier_pred,lgb_classifier_outlier_preds,outlier_pred,average
5868,C_ID_18c1e4935e,0,0.946022,0.955995,0.400544,0.949505,0.813016
14306,C_ID_ca941e5756,0,0.950925,0.943680,0.382554,0.926023,0.800796
15157,C_ID_984c5dad9f,0,0.951979,0.951279,0.410848,0.926393,0.810125
16649,C_ID_6987fbebce,1,0.952310,0.945348,0.374673,0.917262,0.797398
22633,C_ID_768e3cec70,1,0.950869,0.945794,0.394587,0.934947,0.806549
25347,C_ID_0b358310f5,1,0.950066,0.933953,0.469611,0.928543,0.820543
25880,C_ID_ec83c5deb0,1,0.946659,0.931990,0.404966,0.929685,0.803325
32343,C_ID_75b06ef938,0,0.942411,0.940647,0.361684,0.935407,0.795037
38861,C_ID_5e68c3dbab,1,0.944688,0.943415,0.387550,0.931987,0.801910


In [40]:
### testSets
df_temp = pd.read_csv('./classify.csv').sort_values(by=['outlier_rate'],ascending=False)
temp_list = list(df_temp.iloc[:topk,]['card_id'])

outlier_cols = [_f for _f in df_class_test.columns if _f not in ['card_id','target']]
aggs = set(temp_list)

for col in outlier_cols:
    df = df_class_test.sort_values(by=col,ascending=False)[['card_id',col]].iloc[:topk,]
    aggs = set(aggs) & set(df['card_id'])
# aggs = set(aggs) & set(df_Log_sub['card_id'])
print(len(aggs))

40


In [37]:
df_best_sub = pd.read_csv('./submission/df_stackingII_blend_sub.csv')
df_best_sub[df_best_sub['card_id'].isin(list(aggs))]

,card_id,target
496,C_ID_a7d1cd6b7c,-13.125484
5008,C_ID_be92f84f5c,-17.169438
16074,C_ID_9c760806b5,-13.508943
17720,C_ID_938dec7a1a,-13.696203
19114,C_ID_0be3fb41d2,-12.764737
25568,C_ID_767923bdb9,-17.773066
27436,C_ID_493de71141,-15.311665
32446,C_ID_ac114ef831,-18.869935
56780,C_ID_9771b57a38,-12.933803
57626,C_ID_944c62886f,-15.747294


In [24]:
df_best_sub.ix[df_best_sub['card_id'].isin(list(aggs)),['target']] = -33.21928095

In [25]:
df_best_sub.to_csv('./submission/df_best_sub_with_40outliers.csv',index=False)

In [30]:
df = pd.read_csv('./submission/df_best_sub_with_21outliers.csv')

,card_id,target
496,C_ID_a7d1cd6b7c,-33.219281
67890,C_ID_cde027fde7,-33.219281
19114,C_ID_0be3fb41d2,-33.219281
77297,C_ID_b0c41ca140,-33.219281
75740,C_ID_c179e7bb20,-33.219281
56780,C_ID_9771b57a38,-33.219281
17720,C_ID_938dec7a1a,-33.219281
103851,C_ID_877b879853,-33.219281
27436,C_ID_493de71141,-33.219281
108111,C_ID_b789a1772c,-33.219281


In [32]:
df_temp = pd.DataFrame(data=df['card_id'].values,columns=['card_id'])
df_temp['target'] = (df['target'] + df_best_sub['target'])/2

,card_id,target
115682,C_ID_099d9774e2,-33.219281
57626,C_ID_944c62886f,-33.219281
32446,C_ID_ac114ef831,-33.219281
5008,C_ID_be92f84f5c,-33.219281
496,C_ID_a7d1cd6b7c,-33.219281
108111,C_ID_b789a1772c,-33.219281
27436,C_ID_493de71141,-33.219281
103851,C_ID_877b879853,-33.219281
17720,C_ID_938dec7a1a,-33.219281
25568,C_ID_767923bdb9,-33.219281


In [34]:
df_temp.to_csv('./submission/df_average_outliers.csv',index=False)